In [1]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.pipeline import make_pipeline

In [3]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1705076671427, experiment_id='3', last_update_time=1705076671427, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [4]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [5]:
df_train = read_dataframe('../../2.0 model_tracking/data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../../2.0 model_tracking/data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [6]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


2024/01/16 09:55:09 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\AIDOOK~1.ENO\AppData\Local\Temp\tmp7r08inj1\model\model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.3.2', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.
c:\Users\AIDOO K. ENOCH\.conda\envs\model_orch\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [7]:
from mlflow.tracking import MlflowClient


In [8]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = 'b021e7f69b714398b29ca08ee6fd9370'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [9]:
run = client.search_runs(experiment_ids=3)

RUN_ID = run[0].info.run_id
print(RUN_ID)

d37d17bdbd494feca888090b306a0f72


In [10]:
path = mlflow.artifacts.download_artifacts(run_id=RUN_ID, artifact_path="mlflow-artifacts:/3/d37d17bdbd494feca888090b306a0f72/artifacts/model", dst_path='dict_vec/model.pkl')

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'c:\\Users\\AIDOO K. ENOCH\\Documents\\GitHub\\RGT\\MLOPS\\4.0 deployment\\web-service-mlflow\\dict_vec.bin\\mlflow-artifacts:'

In [11]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

C:\Users\AIDOO K. ENOCH\AppData\Local\Temp\ipykernel_18868\2836594815.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')


MlflowException: The following failures occurred while downloading one or more artifacts from http://127.0.0.1:5000/api/2.0/mlflow-artifacts/artifacts/3/d37d17bdbd494feca888090b306a0f72/artifacts: {'dict_vectorizer.bin': 'MlflowException("API request to http://127.0.0.1:5000/api/2.0/mlflow-artifacts/artifacts/3/d37d17bdbd494feca888090b306a0f72/artifacts/dict_vectorizer.bin failed with exception HTTPConnectionPool(host=\'127.0.0.1\', port=5000): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/3/d37d17bdbd494feca888090b306a0f72/artifacts/dict_vectorizer.bin (Caused by ResponseError(\'too many 500 error responses\'))")'}

In [29]:
path = './model.pkl'

In [30]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [14]:
dv

Pipeline(steps=[('dictvectorizer', DictVectorizer()),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=20, min_samples_leaf=10,
                                       n_jobs=-1, random_state=0))])